# 1. 免学部分
* 这一章只有14面内容，所以暂不设免学部分

# 2. 前言

* 大家在做题目前一定要先把第4章的书读一下
* 写本作业，请使用 Anaconda 里的 Notebook
* 我做了一个[打包下载 Link](https://downgit.github.io/#/home?url=https://github.com/liuhui998/pe2/tree/master/04-concat-merge), 大家下载下来解压后，使用 Notebook 打开

# 3. 学习概要
* 我会在每道题有前标明对的书页码
* 如果超纲我会提前提示，并加上资料 Link
---

知识点整理

* pd.concat 合并 DataFrame 数据（非常重要），加上 axis = 1 就是按列合并

* DataFrame 与 Series 数据合并（非常重要），对以要做销售预测的同学非常有帮助

* DataFrame 的成员函数 merge (inner/left) (非常重要），相当于在 Pandas 里实现了 SQL 的功能

  大家以后可以自由的处理大数据
  
  大家一定要看这个 Link: [Pandas Merge 详解](https://blog.csdn.net/stevenkwong/article/details/52540605)，着重看 inner/left, 再做题目。
  
  
* Series 对象里的 isnull() ，检查数据是否为空

* pd.melt 把宽表变成窄表 p114, [Pandas melt 使用](https://blog.csdn.net/maymay_/article/details/80039677)


In [45]:
import pandas as pd

# 4. 合并各个 bu1/bu2/bu3 的费用表

---
备注：这里综合了 85~86面的内容

* 假设大家现在负责统计分析各个 bu 的数据


* 当前目录下有 bu1.xlsx,bu2.xlsx,bu3.xlsx 3个文件，

  将它们分别读取到 df_bu1,df_bu2,df_bu3 中
  
  
* 将 df_bu1,df_bu2,df_bu3 按行合并到 total_df 中

 （备注 pd.concat 是默认按行合并的，所以不用加其它参数）
 
* 统计每个 bu(bu_name) 的费用 (rmb) 汇总 (使用 groupby sum)

* 统计每个 类型（type)的费用 (rmb) 汇总 (使用 groupby sum)

* 画图显示法各个类型费用变化情况 （超纲） （ df.plot.bar() ）
   
  主要是为了复习上一章所学

In [46]:
#当前目录下有 bu1.xlsx,bu2.xlsx,bu3.xlsx 3个文件，
#将它们分别读取到 df_bu1,df_bu2,df_bu3 中
df_bu1 = pd.read_excel('bu1.xlsx')
df_bu2 = pd.read_excel('bu2.xlsx')
df_bu3 = pd.read_excel('bu3.xlsx')


In [47]:
df_bu1.shape

(1000, 4)

In [48]:
#将 df_bu1,df_bu2,df_bu3 按行合并到 total_df 中
total_df = pd.concat([df_bu1,df_bu2,df_bu3])
total_df.shape

(3000, 4)

In [49]:
# 统计每个 bu 的费用 (rmb) 汇总 (使用 groupby sum)
# 方法1
total_df.groupby('bu_name')['rmb'].sum()

bu_name
bu1    971039
bu2    990559
bu3    981853
Name: rmb, dtype: int64

In [50]:
# 统计每个 bu 的费用 (rmb) 汇总 (使用 groupby sum)
# 方法2，可以不指定汇总字段，它会把数字类型都汇总
total_df.groupby('bu_name').sum()

,rmb
bu_name,
bu1,971039
bu2,990559
bu3,981853


In [51]:
#统计每个 类型（type)的费用 (rmb) 汇总 (使用 groupby sum)
# 方法1
total_df.groupby('type')['rmb'].sum()

type
A    228007
B    236375
C    228876
D    235864
E    232074
F    206378
G    229962
H    212301
I    239990
J    214356
K    228146
L    232547
M    218575
Name: rmb, dtype: int64

In [52]:
#统计每个 类型（type)的费用 (rmb) 汇总 (使用 groupby sum)
# 方法2


In [53]:
#画图显示法各个类型费用变化情况 （超纲） （ df.plot.bar() ）
pd.options.plotting.backend = 'plotly'
sum_df = total_df.groupby(['type']).sum()
fig1 = sum_df.plot.bar()
fig1.show()


In [54]:
# 这里再复习了之前的 sort_values 
# 按 rmb 排序后再算图
sorted_sum_df = sum_df.sort_values(by='rmb')
fig2 = sorted_sum_df.plot.bar()
fig2.show()

In [55]:
help(sum_df.sort_values)
# new_fig2 = sum_df.sort_values(by="rmb").plot.bar()
# new_fig2.show()

Help on method sort_values in module pandas.core.frame:

sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last', ignore_index=False) method of pandas.core.frame.DataFrame instance
    Sort by the values along either axis.
    
    Parameters
    ----------
            by : str or list of str
                Name or list of names to sort by.
    
                - if `axis` is 0 or `'index'` then `by` may contain index
                  levels and/or column labels.
                - if `axis` is 1 or `'columns'` then `by` may contain column
                  levels and/or index labels.
    
                .. versionchanged:: 0.23.0
    
                   Allow specifying index or column level names.
    axis : {0 or 'index', 1 or 'columns'}, default 0
         Axis to be sorted.
    ascending : bool or list of bool, default True
         Sort ascending vs. descending. Specify list for multiple sort
         orders.  If this is a list of bools, must

# 5. 处理航空客运量预测数据

---
备注：这里综合了 87面的内容

* 假设你现在穿越到 1959-12月且是澳洲航空管理局的工作人，

   你现在在要负责预测未来1年的客运量的变化，以决定如何增加机场相关的投资

* 现在 Peter 给帮你预测了未来12个月的数据，请把它加数据表中，并画出来图好向领导汇报

* 通过这一题大家可以想像一下如何构造复杂的数据表/图表

---

* 读取 AirPassengers.csv 文件的数据到 air_df 中

* 将 air_df 的 Month 列转成日期类型

* 将 air_df 的 Month 列设为索引

* 将 air_df 的数据画图出来(plotly)

* 新创建一个 Series, 名字为 fcst

  index =  ['1960-01', '1960-02', '1960-03', '1960-04','1960-05', '1960-06', '1960-07', '1960-08', '1960-09', '1960-10', '1960-11', '1960-12']
    
  请将 index 数据类型转成日期型 

  数据部分 = [400, 391, 430, 461, 472, 535, 622, 606, 508, 461, 390, 432]
  
  
* 将 fcst 增加到 air_df 中

* 将 air_df 的数据画图出来(plotly)




In [56]:
#读取 AirPassengers.csv 文件的数据到 air_df 中
air_df = pd.read_csv('AirPassengers.csv')
air_df.head()

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


In [57]:
air_df.dtypes

Month         object
Passengers     int64
dtype: object

In [68]:
#将 air_df 的 Month 列转成日期类型
#https://blog.csdn.net/u010099080/article/details/51769732
from datetime import datetime

month = []
for m in air_df['Month']:
    month.append(datetime.strptime(m,'%Y-%m'))
new_air_df = air_df.copy()
new_air_df.Month = month


In [59]:
new_air_df.dtypes

Month         datetime64[ns]
Passengers             int64
dtype: object

In [69]:
# 将 air_df 的 Month 列设为索引
new_air_df = new_air_df.set_index('Month')
new_air_df.head()

,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [70]:
#将 air_df 的数据画图出来(plotly)
#澳洲空乘客流的变化情况
fig3 = new_air_df.plot()
fig3.show()

In [74]:
#新创建一个 Series, 名字为 fcst
#index = ['1960-01', '1960-02', '1960-03', '1960-04','1960-05', '1960-06', '1960-07', '1960-08', '1960-09', '1960-10', '1960-11', '1960-12']
#请将 index 数据类型转成日期型
#数据部分 = [400, 391, 430, 461, 472, 535, 622, 606, 508, 461, 390, 432]
fsct = pd.Series(
    data=[400, 391, 430, 461, 472, 535, 622, 606, 508, 461, 390, 432],
    index = ['1960-01', '1960-02', '1960-03', '1960-04','1960-05', '1960-06', '1960-07', '1960-08', '1960-09', '1960-10', '1960-11', '1960-12']
)
timeidx = pd.to_datetime(fsct.index,format = '%Y-%m')
fsct.index = timeidx
fsct


1960-01-01    400
1960-02-01    391
1960-03-01    430
1960-04-01    461
1960-05-01    472
1960-06-01    535
1960-07-01    622
1960-08-01    606
1960-09-01    508
1960-10-01    461
1960-11-01    390
1960-12-01    432
dtype: int64

In [76]:
#将 fcst 增加到 air_df 中
fcst_df = pd.concat([new_air_df,fsct])
fcst_df.columns  = ['Passengers','forcast']
fcst_df.tail()

,Passengers,forcast
1960-08-01,NaN,606.0
1960-09-01,NaN,508.0
1960-10-01,NaN,461.0
1960-11-01,NaN,390.0
1960-12-01,NaN,432.0


In [77]:
# 大家仔细看，1960-01 之前的 Fcst 列的数据都是 NaN (为空)
#           对于已发生的数据一般是不预测
# 1960-01 及以后的 Passengers 列的数据都是 NaN (为空)
# 对于未来的数据不知道，所以为空
fcst_df.tail(20)

,Passengers,forcast
1959-05-01,420.0,NaN
1959-06-01,472.0,NaN
1959-07-01,548.0,NaN
1959-08-01,559.0,NaN
1959-09-01,463.0,NaN
1959-10-01,407.0,NaN
1959-11-01,362.0,NaN
1959-12-01,405.0,NaN
1960-01-01,NaN,400.0
1960-02-01,NaN,391.0


In [78]:
# 将 air_df 的数据画图出来(plotly)
# Plotly 很聪明把现在(Passengers) 和 未来(Fcst) 的数据分开了
fig5 = fcst_df.plot()
fig5.show()

# 6. 合并历年 BU 的销量数量
---
备注：这里综合了 89面的内容

* 超纲 pd.melt 把宽表变成窄表 p114, [Pandas melt 使用](https://blog.csdn.net/maymay_/article/details/80039677)


* 假设大家现在负责统计分析各个 bu 的销量数量
* 近3年的销量汇总数据放在三张 Excel 表中
* 第1列是bu_name, 剩下的列是年数

---
* 当前目录下有 total1.xlsx, total2.xlsx, total3.xlsx 3个文件，

  将它们分别读取到 total1,total2,total3 中


* 将 total1,total2,total3 按列合并到 total 中

* 尝试将 total 的宽表变成窄表



In [104]:
#当前目录下有 total1.xlsx, total2.xlsx, total3.xlsx 3个文件，
#  将它们分别读取到 total1,total2,total3 中
total1 = pd.read_excel('total1.xlsx')
total1 = total1.set_index('bu_name')

In [105]:
total2 = pd.read_excel('total2.xlsx')
total2 = total2.set_index('bu_name')

In [106]:
total3 = pd.read_excel('total3.xlsx')
total3 = total3.set_index('bu_name')

In [109]:
#将 total1,total2,total3 按列合并到 total 中
total = pd.concat([total1,total2,total3],axis=1)
total = total.reset_index()
total

,bu_name,2010,2011,2012
0,bu1,45420820,41627609,56730549
1,bu2,45198851,43769878,57333429
2,bu3,44874696,43981131,57025416
3,bu4,45402272,44191673,57516801
4,bu5,45265149,44125129,57303312
5,bu6,45243900,43910332,56926536
6,bu7,45450858,43838085,56961676
7,bu8,45136369,44059557,56714264
8,bu9,44792492,44111467,56947196
9,buX,44797543,43930000,57502649


In [116]:
#尝试将 total 的宽表变成窄表
new_total = pd.melt(total,
       id_vars="bu_name",
       var_name="year",
       value_name="sales")
new_total

,bu_name,year,sales
0,bu1,2010,45420820
1,bu2,2010,45198851
2,bu3,2010,44874696
3,bu4,2010,45402272
4,bu5,2010,45265149
5,bu6,2010,45243900
6,bu7,2010,45450858
7,bu8,2010,45136369
8,bu9,2010,44792492
9,buX,2010,44797543


In [118]:
# 看每年销量的变化（额外）
new_total_df = new_total.groupby('year').sum()
fig6 = new_total_df.plot()
fig6.show()
new_total_df

,sales
year,
2010,451582950
2011,437544861
2012,570961828


# 7. 像专家一样，操作数据
---
备注：这里综合了 93～97 面的内容

* isnull() 是98，99 面的内容

* [Pandas Merge 详解](https://blog.csdn.net/stevenkwong/article/details/52540605)，着重看 inner/left

  这个一定一定一定一定一定要认真看，对于大家处理复杂数据会非常有帮助，程序员数据分析师就这样处理数据的
---
数据介绍

* 商品表(products.xlsx)，里面有 sku, name  字段
* 订单表(orders.xlsx), 里面有 order_no(订单号),sku,quantity 字段
* 物流表(trackings.xlsx) 里面有 order_no(订单号),tracking_no(快递号), is_delivered(是否已配送成功) 字段

---
题目要求

* 将 products.xlsx 读到 products
* 将 orders.xlsx 读取 orders
* 将 trackings.xlsx 读取到 trackings

* 生成 orders1, 根据 orders/products 的sku 相关联，为其增加 name 字段

  使用 merge 函数
  
    
* 生成 orders2, 根据 orders/trackings 的sku 相关联，为其增加 tracking_no，is_delivered 字段

  使用 merge 函数，how="left"
  

* 找出所有已配送成功功的订单

* 找出所有没物流信息的订单



In [120]:
products = pd.read_excel('products.xlsx')
products

,sku,name
0,A1234,AirPods
1,A1235,MacBookPro


In [119]:
orders = pd.read_excel('orders.xlsx')
orders

,order_no,sku,quantity
0,O_2020_01,A1234,2
1,O_2020_02,A1234,2
2,O_2020_03,A1235,1
3,O_2020_04,A1235,1


In [121]:
trackings = pd.read_excel('trackings.xlsx')
trackings

,order_no,tracking_no,is_delivered
0,O_2020_01,T_2020_1,1
1,O_2020_02,T_2020_2,0
2,O_2020_03,T_2020_3,1


In [126]:
#生成 orders1, 根据 orders/products 的sku 相关联，为其增加 name 字段
#使用 merge 函数
  
order1 = pd.merge(orders,products,on=['sku','sku'],how='left')
order1

,order_no,sku,quantity,name
0,O_2020_01,A1234,2,AirPods
1,O_2020_02,A1234,2,AirPods
2,O_2020_03,A1235,1,MacBookPro
3,O_2020_04,A1235,1,MacBookPro


In [127]:
# 第二种方法
order1 = orders.merge(products,left_on="sku",right_on="sku",how="inner")
order1

,order_no,sku,quantity,name
0,O_2020_01,A1234,2,AirPods
1,O_2020_02,A1234,2,AirPods
2,O_2020_03,A1235,1,MacBookPro
3,O_2020_04,A1235,1,MacBookPro


In [128]:
# 第三种方法
order1 = orders.merge(products)
order1

,order_no,sku,quantity,name
0,O_2020_01,A1234,2,AirPods
1,O_2020_02,A1234,2,AirPods
2,O_2020_03,A1235,1,MacBookPro
3,O_2020_04,A1235,1,MacBookPro


In [129]:
#生成 orders2, 根据 orders/trackings order_no 相关联，为其增加 tracking_no，is_delivered 字段
#使用 merge 函数，how="left"
order2 = orders.merge(trackings,on=['order_no'],how='left')
order2

,order_no,sku,quantity,tracking_no,is_delivered
0,O_2020_01,A1234,2,T_2020_1,1.0
1,O_2020_02,A1234,2,T_2020_2,0.0
2,O_2020_03,A1235,1,T_2020_3,1.0
3,O_2020_04,A1235,1,NaN,NaN


In [130]:
#找出所有已配送成功功的订单
order3 = order2[order2['is_delivered'] == 1]
order3

,order_no,sku,quantity,tracking_no,is_delivered
0,O_2020_01,A1234,2,T_2020_1,1.0
2,O_2020_03,A1235,1,T_2020_3,1.0


In [132]:
#找出所有没物流信息的订单
order4 = order2[order2.tracking_no.isnull()]
order4

,order_no,sku,quantity,tracking_no,is_delivered
3,O_2020_04,A1235,1,NaN,NaN


In [133]:
order2.tracking_no.isnull()

0    False
1    False
2    False
3     True
Name: tracking_no, dtype: bool

# 8. 参考资料
* [Pandas Merge 详解](https://blog.csdn.net/stevenkwong/article/details/52540605)，着重看 inner/left
* pd.melt 把宽表变成窄表  [Pandas melt 使用](https://blog.csdn.net/maymay_/article/details/80039677)
* [Python 中 strptime 的简单使用](https://blog.csdn.net/u010099080/article/details/51769732)